In [1]:
#필요 라이브러리 import
import numpy as np
import pandas as pd

## 1. 데이터 불러오기

In [58]:
#휴게음식점 데이터 불러오기
df1 = pd.read_csv('휴게음식점.csv', encoding='cp949')
df1.shape

/var/folders/jc/q2j97l_x5hb7jh99pn7nsbym0000gn/T/ipykernel_628/1310492345.py:2: DtypeWarning: Columns (11,39,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('휴게음식점.csv', encoding='cp949')


(550150, 48)

In [59]:
#일반음식점 데이터 불러오기
df2 = pd.read_csv('일반음식점.csv', encoding='cp949')
df2.shape

/var/folders/jc/q2j97l_x5hb7jh99pn7nsbym0000gn/T/ipykernel_628/2418469537.py:2: DtypeWarning: Columns (11,27,39,41,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('일반음식점.csv', encoding='cp949')


(2105706, 48)

## 2. 데이터 살펴보기

- 사용할 변수 확인
- 추출해야할 카테고리 확인

In [60]:
#열(속성)확인하기(df1, df2는 같은 형식으로 작성되어 있음)
df1.columns

Index(['번호', '개방서비스명', '개방서비스아이디', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자',
       '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자',
       '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '소재지전체주소', '도로명전체주소', '도로명우편번호',
       '사업장명', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보(x)', '좌표정보(y)',
       '위생업태명', '남성종사자수', '여성종사자수', '영업장주변구분명', '등급구분명', '급수시설구분명', '총직원수',
       '본사직원수', '공장사무직직원수', '공장판매직직원수', '공장생산직직원수', '건물소유구분명', '보증액', '월세액',
       '다중이용업소여부', '시설총규모', '전통업소지정번호', '전통업소주된음식', '홈페이지', 'Unnamed: 47'],
      dtype='object')

In [61]:
#df1의 업태구문명이 무엇이 얼마나 있는지 확인
df1['위생업태명'].value_counts()

위생업태명
커피숍         162542
기타 휴게음식점     92095
일반조리판매       86761
다방           79598
편의점          51228
패스트푸드        30685
과자점          23429
백화점           6616
푸드트럭          6223
전통찻집          2693
아이스크림         2633
키즈카페          1287
고속도로          1245
떡카페           1081
철도역구내          753
유원지            682
관광호텔           250
극장             212
공항              84
한식              11
분식               8
기타               4
김밥(도시락)          2
호프/통닭            2
룸살롱              1
식품소분업            1
까페               1
경양식              1
식육(숯불구이)         1
단란주점             1
Name: count, dtype: int64

In [62]:
#df2의 업태구문명이 무엇이 얼마나 있는지 확인
df2['위생업태명'].value_counts()

위생업태명
한식                 967555
기타                 228009
호프/통닭              202207
분식                 178829
경양식                119819
식육(숯불구이)            68535
중국식                 68131
통닭(치킨)              55049
일식                  51962
정종/대포집/소주방          43918
횟집                  31038
까페                  22160
패스트푸드               18734
뷔페식                 10393
탕류(보신용)             10029
김밥(도시락)              9094
외국음식전문점(인도,태국등)      7988
전통찻집                 2278
라이브카페                1582
패밀리레스트랑              1580
냉면집                  1453
복어취급                 1433
출장조리                 1120
감성주점                 1088
이동조리                  904
키즈카페                  628
일반조리판매                  9
기타 휴게음식점                7
커피숍                     6
간이주점                    4
다방                      2
식품소분업                   1
제과점영업                   1
식품등 수입판매업               1
룸살롱                     1
Name: count, dtype: int64

- 사용할 속성
    - 사업장명
    - 영업상태명
    - 인허가일자
    - 폐업일자
    - 소재지면적
    - 소재지전체주소
    - 위생업태명
- df1에서 추출한 업태구문명
    - 커피숍
    - 다방
    - 전통찻집
    - 떡카페
    - 까페
- df2에서 추출한 업태구분명
    - 까페
    - 전통찻집
    - 커피숍
    - 다방

## 3. 데이터 전처리
    1. 기존 데이터에서 필요한 카테고리 내 데이터만 추출 및 결측치 제거
    2. 추출 데이터 중 서울에 존재하는 데이터만 추출
    3. '한시적' 운영 카페 삭제(분석 대상에 포함되지 않음)
    4. 이상데이터 삭제(카페 분류에 치킨, 피자 등의 상호명 확인 후 제거)
    5. 면적 결측치 존재하는 데이터 삭제
    
    이상 데이터 삭제하기
    - '맥주', '주점', '호프', '펍', '치킨', '스시', '국수', '만두', '닭', '떡볶이', '족발', '김밥', '죽', '면'

### (1) df1 전처리

In [63]:
#1. df1에서 커피전문점 데이터만 추출하기
df1_cafe = df1[(df1['위생업태명'] == '커피숍') | (df1['위생업태명'] == '다방') | (df1['위생업태명'] == '전통찻집') | (df1['위생업태명'] == '떡카페') | (df1['위생업태명'] == '까페')]
df1_cafe = df1_cafe[['사업장명','영업상태명', '인허가일자', '폐업일자', '소재지면적','소재지전체주소', '위생업태명']]
df1_cafe

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명
6,덕소낙원떡집,폐업,2023-09-21,2023-09-22,39.90,경기도 남양주시 와부읍 덕소리 488-4,떡카페
19,카페지와이(고고마켓),폐업,2023-09-22,2023-09-24,NaN,울산광역시 중구 태화동 667 태화강 국가정원 안내센터,커피숍
44,위드고,폐업,2023-09-22,2023-10-08,NaN,서울특별시 노원구 상계동 713 롯데백화점,커피숍
49,쿠키앤모어,폐업,2023-09-22,2023-10-21,3.30,경기도 화성시 반송동 96 메타폴리스,커피숍
51,(주)제이와이에스유통,폐업,2023-09-22,2023-10-12,6.60,경기도 고양시 일산동구 장항동 784 롯데백화점일산점,커피숍
...,...,...,...,...,...,...,...
550134,스타벅스동대구터미널점,영업/정상,2011-08-11,NaN,252.00,"대구광역시 동구 신천동 330-6 번지외1필지 보광빌딩 가동 1,2층",커피숍
550135,메가엠지씨커피 광주봉선유안공원점,영업/정상,2008-08-14,NaN,65.00,광주광역시 남구 봉선동 138-2 (1층)103호,커피숍
550138,파운데이 염주점,영업/정상,2010-05-28,NaN,125.74,광주광역시 서구 화정동 824-17 (1층),커피숍
550141,벌크커피봉선점,영업/정상,2017-10-23,NaN,35.00,광주광역시 남구 봉선동 606번지 남양휴턴1차 106동 107호,커피숍


In [64]:
#df1_cafe 소재지 결측이 있는 데이터 제거
df1_cafe2 = df1_cafe.dropna(subset=['소재지전체주소'], how='any', axis=0)

In [65]:
#2. df1_cafe2에서 서울특별시에 위치하는 데이터 추출
total_cafe1 = df1_cafe2[df1_cafe2['소재지전체주소'].str.contains('서울특별시')]
total_cafe1

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명
44,위드고,폐업,2023-09-22,2023-10-08,NaN,서울특별시 노원구 상계동 713 롯데백화점,커피숍
244,텐퍼센트커피(Tenpercent coffee),폐업,2023-09-18,2023-09-25,53.00,서울특별시 노원구 공릉동 567-4,커피숍
305,커피크루,폐업,2023-09-27,2023-10-06,NaN,서울특별시 서초구 양재동 232 AT센터,커피숍
309,마이빈(한시적),폐업,2023-10-06,2023-10-18,NaN,서울특별시 강남구 삼성동 159 코엑스,커피숍
331,엔젤리너스커피영등포점,폐업,2023-10-24,2023-10-29,6.60,서울특별시 영등포구 여의도동 84-4 여의도한강공원 이벤트광장,커피숍
...,...,...,...,...,...,...,...
541185,굿애프터눈 합정점,영업/정상,2017-04-10,NaN,50.42,서울특별시 마포구 합정동 473 마포한강2차푸르지오상업시설 120호,커피숍
541187,보노보나마포합정점,영업/정상,2017-04-18,NaN,41.53,서울특별시 마포구 합정동 473번지,커피숍
541188,셀렉토커피 상암누리꿈스퀘어점,영업/정상,2017-04-14,NaN,43.76,서울특별시 마포구 상암동 1605 누리꿈스퀘어 지하1층 B1087호,커피숍
541190,로벤,영업/정상,2016-11-18,NaN,23.00,서울특별시 마포구 서교동 368-11번지,커피숍


In [66]:
#3. 사업장명에 (한시적)인 변수 확인
delete_limited1 = total_cafe1[total_cafe1['사업장명'].str.contains('(한시적)')]
delete_limited1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 122 entries, 309 to 348794
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     122 non-null    object 
 1   영업상태명    122 non-null    object 
 2   인허가일자    122 non-null    object 
 3   폐업일자     119 non-null    object 
 4   소재지면적    30 non-null     float64
 5   소재지전체주소  122 non-null    object 
 6   위생업태명    122 non-null    object 
dtypes: float64(1), object(6)
memory usage: 7.6+ KB


/var/folders/jc/q2j97l_x5hb7jh99pn7nsbym0000gn/T/ipykernel_628/2157973218.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  delete_limited1 = total_cafe1[total_cafe1['사업장명'].str.contains('(한시적)')]


In [67]:
#사업장명에 (한시적)인 변수 제거
total_cafe11 = total_cafe1.copy()
total_cafe11.drop(total_cafe11[total_cafe11['사업장명'].str.contains('(한시적)')].index, inplace=True)
total_cafe11

/var/folders/jc/q2j97l_x5hb7jh99pn7nsbym0000gn/T/ipykernel_628/4258378861.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  total_cafe11.drop(total_cafe11[total_cafe11['사업장명'].str.contains('(한시적)')].index, inplace=True)


,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명
44,위드고,폐업,2023-09-22,2023-10-08,NaN,서울특별시 노원구 상계동 713 롯데백화점,커피숍
244,텐퍼센트커피(Tenpercent coffee),폐업,2023-09-18,2023-09-25,53.00,서울특별시 노원구 공릉동 567-4,커피숍
305,커피크루,폐업,2023-09-27,2023-10-06,NaN,서울특별시 서초구 양재동 232 AT센터,커피숍
331,엔젤리너스커피영등포점,폐업,2023-10-24,2023-10-29,6.60,서울특별시 영등포구 여의도동 84-4 여의도한강공원 이벤트광장,커피숍
766,트래쉬버스터즈 스낵바,폐업,2023-10-12,2023-10-26,36.00,서울특별시 강남구 대치동 514 SETEC,커피숍
...,...,...,...,...,...,...,...
541185,굿애프터눈 합정점,영업/정상,2017-04-10,NaN,50.42,서울특별시 마포구 합정동 473 마포한강2차푸르지오상업시설 120호,커피숍
541187,보노보나마포합정점,영업/정상,2017-04-18,NaN,41.53,서울특별시 마포구 합정동 473번지,커피숍
541188,셀렉토커피 상암누리꿈스퀘어점,영업/정상,2017-04-14,NaN,43.76,서울특별시 마포구 상암동 1605 누리꿈스퀘어 지하1층 B1087호,커피숍
541190,로벤,영업/정상,2016-11-18,NaN,23.00,서울특별시 마포구 서교동 368-11번지,커피숍


In [68]:
#total_cafe11 면적 결측이 있는 데이터 제거
total_cafe12 = total_cafe11.dropna(subset=['소재지면적'], how='any', axis=0)
total_cafe12

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명
244,텐퍼센트커피(Tenpercent coffee),폐업,2023-09-18,2023-09-25,53.00,서울특별시 노원구 공릉동 567-4,커피숍
331,엔젤리너스커피영등포점,폐업,2023-10-24,2023-10-29,6.60,서울특별시 영등포구 여의도동 84-4 여의도한강공원 이벤트광장,커피숍
766,트래쉬버스터즈 스낵바,폐업,2023-10-12,2023-10-26,36.00,서울특별시 강남구 대치동 514 SETEC,커피숍
1047,카라멜리에오,폐업,2023-10-26,2023-10-31,12.23,서울특별시 강남구 압구정동 429 현대백화점본점,커피숍
1091,담은,폐업,2023-09-14,2023-10-24,56.00,서울특별시 강남구 도곡동 419-6,커피숍
...,...,...,...,...,...,...,...
541185,굿애프터눈 합정점,영업/정상,2017-04-10,NaN,50.42,서울특별시 마포구 합정동 473 마포한강2차푸르지오상업시설 120호,커피숍
541187,보노보나마포합정점,영업/정상,2017-04-18,NaN,41.53,서울특별시 마포구 합정동 473번지,커피숍
541188,셀렉토커피 상암누리꿈스퀘어점,영업/정상,2017-04-14,NaN,43.76,서울특별시 마포구 상암동 1605 누리꿈스퀘어 지하1층 B1087호,커피숍
541190,로벤,영업/정상,2016-11-18,NaN,23.00,서울특별시 마포구 서교동 368-11번지,커피숍


In [69]:
#이상데이터 확인
total_cafe_check1 = total_cafe12[(total_cafe12['사업장명'].str.contains('맥주')) | (total_cafe12['사업장명'].str.contains('주점'))| (total_cafe12['사업장명'].str.contains('호프'))| (total_cafe12['사업장명'].str.contains('펍'))| (total_cafe12['사업장명'].str.contains('맥주'))| (total_cafe12['사업장명'].str.contains('치킨'))| (total_cafe12['사업장명'].str.contains('스시')) | (total_cafe12['사업장명'].str.contains('국수'))| (total_cafe12['사업장명'].str.contains('만두'))| (total_cafe12['사업장명'].str.contains('닭'))| (total_cafe12['사업장명'].str.contains('떡볶이'))| (total_cafe12['사업장명'].str.contains('김밥'))| (total_cafe12['사업장명'].str.contains('족발'))| (total_cafe12['사업장명'].str.contains('죽'))].index
total_cafe_check1

Index([  9222,   9525,  41725,  42218,  43413,  44486,  46104,  47430,  48975,
        49507,  50774,  54010,  56615,  92264,  93999,  98047,  98788, 123157,
       130049, 132376, 151858, 152310, 153456, 153827, 154879, 156291, 157725,
       157766, 159371, 162382, 162467, 164868, 166342, 167851, 171935, 173288,
       173612, 211984, 215460, 217028, 223888, 224177, 224337, 226606, 233324,
       260862, 263311, 266006, 267004, 267211, 267352, 268290, 268326, 268481,
       270876, 272335, 272908, 273607, 273802, 353068, 355495, 377642, 385989,
       408666, 422074, 438611, 445354, 448227, 448444, 448967, 455338, 467660,
       468423, 469042, 470009, 470630, 471840, 490981, 504946, 511463, 512842,
       513093, 520011, 520911, 540986],
      dtype='int64')

In [70]:
#이상 데이터 삭제
df_cafe_1 = total_cafe12.copy()
df_cafe_1.drop(total_cafe_check1, inplace=True)
df_cafe_1

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명
244,텐퍼센트커피(Tenpercent coffee),폐업,2023-09-18,2023-09-25,53.00,서울특별시 노원구 공릉동 567-4,커피숍
331,엔젤리너스커피영등포점,폐업,2023-10-24,2023-10-29,6.60,서울특별시 영등포구 여의도동 84-4 여의도한강공원 이벤트광장,커피숍
766,트래쉬버스터즈 스낵바,폐업,2023-10-12,2023-10-26,36.00,서울특별시 강남구 대치동 514 SETEC,커피숍
1047,카라멜리에오,폐업,2023-10-26,2023-10-31,12.23,서울특별시 강남구 압구정동 429 현대백화점본점,커피숍
1091,담은,폐업,2023-09-14,2023-10-24,56.00,서울특별시 강남구 도곡동 419-6,커피숍
...,...,...,...,...,...,...,...
541185,굿애프터눈 합정점,영업/정상,2017-04-10,NaN,50.42,서울특별시 마포구 합정동 473 마포한강2차푸르지오상업시설 120호,커피숍
541187,보노보나마포합정점,영업/정상,2017-04-18,NaN,41.53,서울특별시 마포구 합정동 473번지,커피숍
541188,셀렉토커피 상암누리꿈스퀘어점,영업/정상,2017-04-14,NaN,43.76,서울특별시 마포구 상암동 1605 누리꿈스퀘어 지하1층 B1087호,커피숍
541190,로벤,영업/정상,2016-11-18,NaN,23.00,서울특별시 마포구 서교동 368-11번지,커피숍


### (2) df2 전처리

In [71]:
#1. df1에서 커피전문점 데이터만 추출하기
df2_cafe = df2[(df2['위생업태명'] == '까페') | (df2['위생업태명'] == '전통찻집') | (df2['위생업태명'] == '커피숍') | (df2['위생업태명'] == '다방')]
df2_cafe = df2_cafe[['사업장명','영업상태명', '인허가일자', '폐업일자', '소재지면적','소재지전체주소', '위생업태명']]
df2_cafe

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명
591,시청역,폐업,2023-10-18,2023-10-22,NaN,충청남도 논산시 강경읍 황산리 198,까페
2259,미엔,폐업,2005-07-04,2023-09-25,34.00,대구광역시 달서구 상인동 1419-6,까페
4100,벤티,폐업,2011-03-25,2023-06-30,85.00,전라남도 곡성군 곡성읍 읍내리 327-1,까페
6279,홍익까페,폐업,2015-03-31,2022-12-12,34.99,경상북도 포항시 북구 죽도동 44-10,까페
8006,나이스웨더 AK분당점,폐업,2020-11-27,2022-07-15,13.96,경기도 성남시 분당구 서현동 263 AK플라자 1층 일부,까페
...,...,...,...,...,...,...,...
2105259,옴서감서,영업/정상,1992-03-13,NaN,22.00,광주광역시 남구 주월동 1278-4번지 (일층),전통찻집
2105369,카메,영업/정상,2010-09-29,NaN,78.00,광주광역시 남구 봉선동 107-13 (1층),까페
2105608,투썸플레이스광주월곡점,영업/정상,1992-10-13,NaN,375.20,광주광역시 광산구 월곡동 313-4,까페
2105646,카페515(cafe 515),영업/정상,2013-02-20,NaN,64.13,광주광역시 남구 양림동 72-3 (1층),까페


In [72]:
#df2_cafe 소재지 결측이 있는 데이터 제거
df2_cafe2 = df2_cafe.dropna(subset=['소재지전체주소'], how='any', axis=0)
df2_cafe2

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명
591,시청역,폐업,2023-10-18,2023-10-22,NaN,충청남도 논산시 강경읍 황산리 198,까페
2259,미엔,폐업,2005-07-04,2023-09-25,34.00,대구광역시 달서구 상인동 1419-6,까페
4100,벤티,폐업,2011-03-25,2023-06-30,85.00,전라남도 곡성군 곡성읍 읍내리 327-1,까페
6279,홍익까페,폐업,2015-03-31,2022-12-12,34.99,경상북도 포항시 북구 죽도동 44-10,까페
8006,나이스웨더 AK분당점,폐업,2020-11-27,2022-07-15,13.96,경기도 성남시 분당구 서현동 263 AK플라자 1층 일부,까페
...,...,...,...,...,...,...,...
2105259,옴서감서,영업/정상,1992-03-13,NaN,22.00,광주광역시 남구 주월동 1278-4번지 (일층),전통찻집
2105369,카메,영업/정상,2010-09-29,NaN,78.00,광주광역시 남구 봉선동 107-13 (1층),까페
2105608,투썸플레이스광주월곡점,영업/정상,1992-10-13,NaN,375.20,광주광역시 광산구 월곡동 313-4,까페
2105646,카페515(cafe 515),영업/정상,2013-02-20,NaN,64.13,광주광역시 남구 양림동 72-3 (1층),까페


In [73]:
#2. df1_cafe2에서 서울특별시에 위치하는 데이터 추출
total_cafe2 = df2_cafe2[df2_cafe2['소재지전체주소'].str.contains('서울특별시')]
total_cafe2

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명
10031,바론,폐업,2015-02-04,2020-02-28,19.00,서울특별시 구로구 개봉동 139-20번지,까페
13504,럼프슈가(Lump Sugar),폐업,2014-10-24,2020-02-07,30.00,서울특별시 동대문구 이문동 264-167번지 1층,까페
13505,멜팅,폐업,2014-10-23,2016-11-07,10.80,서울특별시 동대문구 답십리동 178번지 래미안위브 1층 상가동 2-6호,까페
13759,이빠이,폐업,1999-12-03,2005-05-20,46.40,서울특별시 마포구 서교동 361-10번지,까페
13812,데일리,폐업,2014-11-24,2018-01-05,30.00,서울특별시 동대문구 장안동 216-32번지 1층,까페
...,...,...,...,...,...,...,...
2061332,갈채,영업/정상,1986-04-25,NaN,19.89,서울특별시 성북구 석관동 71-8,까페
2061422,블루씨티,영업/정상,2001-05-23,NaN,85.80,서울특별시 은평구 응암동 84-32 지하1층,까페
2061463,예스호프,영업/정상,2007-04-05,NaN,26.00,서울특별시 양천구 신월동 237-20 1층,까페
2061469,유토피아 카페,영업/정상,2014-10-30,NaN,167.32,서울특별시 은평구 갈현동 454-3번지 2층,까페


In [74]:
#3. 사업장명에 (한시적)인 변수 확인
delete_limited2 = total_cafe2[total_cafe2['사업장명'].str.contains('한시적')]
delete_limited2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     0 non-null      object 
 1   영업상태명    0 non-null      object 
 2   인허가일자    0 non-null      object 
 3   폐업일자     0 non-null      object 
 4   소재지면적    0 non-null      float64
 5   소재지전체주소  0 non-null      object 
 6   위생업태명    0 non-null      object 
dtypes: float64(1), object(6)
memory usage: 0.0+ bytes


In [75]:
#total_cafe11 면적 결측이 있는 데이터 제거
total_cafe2 = total_cafe2.dropna(subset=['소재지면적'], how='any', axis=0)
total_cafe2

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명
10031,바론,폐업,2015-02-04,2020-02-28,19.00,서울특별시 구로구 개봉동 139-20번지,까페
13504,럼프슈가(Lump Sugar),폐업,2014-10-24,2020-02-07,30.00,서울특별시 동대문구 이문동 264-167번지 1층,까페
13505,멜팅,폐업,2014-10-23,2016-11-07,10.80,서울특별시 동대문구 답십리동 178번지 래미안위브 1층 상가동 2-6호,까페
13759,이빠이,폐업,1999-12-03,2005-05-20,46.40,서울특별시 마포구 서교동 361-10번지,까페
13812,데일리,폐업,2014-11-24,2018-01-05,30.00,서울특별시 동대문구 장안동 216-32번지 1층,까페
...,...,...,...,...,...,...,...
2061332,갈채,영업/정상,1986-04-25,NaN,19.89,서울특별시 성북구 석관동 71-8,까페
2061422,블루씨티,영업/정상,2001-05-23,NaN,85.80,서울특별시 은평구 응암동 84-32 지하1층,까페
2061463,예스호프,영업/정상,2007-04-05,NaN,26.00,서울특별시 양천구 신월동 237-20 1층,까페
2061469,유토피아 카페,영업/정상,2014-10-30,NaN,167.32,서울특별시 은평구 갈현동 454-3번지 2층,까페


In [76]:
#4. 이상데이터 확인
total_cafe_check2 = total_cafe2[(total_cafe2['사업장명'].str.contains('맥주')) | (total_cafe2['사업장명'].str.contains('주점'))| (total_cafe2['사업장명'].str.contains('호프'))| (total_cafe2['사업장명'].str.contains('펍'))| (total_cafe2['사업장명'].str.contains('치킨'))| (total_cafe2['사업장명'].str.contains('스시'))| (total_cafe2['사업장명'].str.contains('국수'))| (total_cafe2['사업장명'].str.contains('만두'))| (total_cafe2['사업장명'].str.contains('닭'))| (total_cafe2['사업장명'].str.contains('떢볶이'))| (total_cafe2['사업장명'].str.contains('족발')) | (total_cafe2['사업장명'].str.contains('김밥'))| (total_cafe2['사업장명'].str.contains('죽'))].index
total_cafe_check2

Index([  13855,   16463,   16768,   16960,  140339,  141338,  141673,  141789,
        142642,  144772,
       ...
       1932847, 1932978, 1933620, 1933869, 1934248, 1934664, 1943802, 1999944,
       2001375, 2061463],
      dtype='int64', length=145)

In [77]:
#이상데이터 삭제
df_cafe_2 = total_cafe2.copy()
df_cafe_2.drop(total_cafe_check2, inplace=True)
df_cafe_2

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명
10031,바론,폐업,2015-02-04,2020-02-28,19.00,서울특별시 구로구 개봉동 139-20번지,까페
13504,럼프슈가(Lump Sugar),폐업,2014-10-24,2020-02-07,30.00,서울특별시 동대문구 이문동 264-167번지 1층,까페
13505,멜팅,폐업,2014-10-23,2016-11-07,10.80,서울특별시 동대문구 답십리동 178번지 래미안위브 1층 상가동 2-6호,까페
13759,이빠이,폐업,1999-12-03,2005-05-20,46.40,서울특별시 마포구 서교동 361-10번지,까페
13812,데일리,폐업,2014-11-24,2018-01-05,30.00,서울특별시 동대문구 장안동 216-32번지 1층,까페
...,...,...,...,...,...,...,...
2061319,구할 구,영업/정상,1970-10-14,NaN,98.66,서울특별시 중구 남산동2가 24-13 (1층2층),까페
2061332,갈채,영업/정상,1986-04-25,NaN,19.89,서울특별시 성북구 석관동 71-8,까페
2061422,블루씨티,영업/정상,2001-05-23,NaN,85.80,서울특별시 은평구 응암동 84-32 지하1층,까페
2061469,유토피아 카페,영업/정상,2014-10-30,NaN,167.32,서울특별시 은평구 갈현동 454-3번지 2층,까페


### (3) 데이터 합치기

In [78]:
total_cafe_df = pd.concat([df_cafe_1, df_cafe_2])
total_cafe_df

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명
244,텐퍼센트커피(Tenpercent coffee),폐업,2023-09-18,2023-09-25,53.00,서울특별시 노원구 공릉동 567-4,커피숍
331,엔젤리너스커피영등포점,폐업,2023-10-24,2023-10-29,6.60,서울특별시 영등포구 여의도동 84-4 여의도한강공원 이벤트광장,커피숍
766,트래쉬버스터즈 스낵바,폐업,2023-10-12,2023-10-26,36.00,서울특별시 강남구 대치동 514 SETEC,커피숍
1047,카라멜리에오,폐업,2023-10-26,2023-10-31,12.23,서울특별시 강남구 압구정동 429 현대백화점본점,커피숍
1091,담은,폐업,2023-09-14,2023-10-24,56.00,서울특별시 강남구 도곡동 419-6,커피숍
...,...,...,...,...,...,...,...
2061319,구할 구,영업/정상,1970-10-14,NaN,98.66,서울특별시 중구 남산동2가 24-13 (1층2층),까페
2061332,갈채,영업/정상,1986-04-25,NaN,19.89,서울특별시 성북구 석관동 71-8,까페
2061422,블루씨티,영업/정상,2001-05-23,NaN,85.80,서울특별시 은평구 응암동 84-32 지하1층,까페
2061469,유토피아 카페,영업/정상,2014-10-30,NaN,167.32,서울특별시 은평구 갈현동 454-3번지 2층,까페


### (4) 프랜차이즈 구분
논문을 인용해 공정거래위원회 가맹사업거래사이트에서 정보공개서 비교정보를 이용해 서울에 직영점 또는 가맹점/직영점이 30개 이상 있는 프랜차이즈를 검색했고 직영점위주로 가맹사업거래사이트에서 검색되지 않는 대형 프랜차이즈도 포함시켜 총 n개의 사업체를 프랜차이즈로 처리하기로 결정했다. 

- 이디야커피
- 메가엠지씨커피
- 투썸플레이스
- 컴포즈커피
- 빽다방
- 더벤티
- 커피베이
- 커피에 반하다
- 요거프레소
- 파스쿠찌
- 할리스/할리스커피
- 엔제리너스
- 디저트39
- 탐앤탐스커피
- 매머드
- 감성커피
- 달리는커피
- 카페베네
- 셀렉토커피
- 토프레소
- 더카페 !!
- 아마스빈
- 만랩커피
- 더치앤빈
- 카페게이트
- 달.콤
- 커피사피엔스
- 커스텀커피
- 커피나무
- 커피빈(70)
- 스타벅스(611)
- 아티제(53)
- 폴바셋(64)
- 커피스미스(44)
- 공차

In [79]:
#프랜차이즈여부 변수 생성 1 : 프랜차이즈, 0 : 독립
total_cafe_df['프랜차이즈'] = 0
total_cafe_df.tail()

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명,프랜차이즈
2061319,구할 구,영업/정상,1970-10-14,NaN,98.66,서울특별시 중구 남산동2가 24-13 (1층2층),까페,0
2061332,갈채,영업/정상,1986-04-25,NaN,19.89,서울특별시 성북구 석관동 71-8,까페,0
2061422,블루씨티,영업/정상,2001-05-23,NaN,85.80,서울특별시 은평구 응암동 84-32 지하1층,까페,0
2061469,유토피아 카페,영업/정상,2014-10-30,NaN,167.32,서울특별시 은평구 갈현동 454-3번지 2층,까페,0
2061604,꽃샘,영업/정상,1983-07-19,NaN,20.80,서울특별시 성북구 길음동 1257-2번지,까페,0


In [80]:
#프랜차이즈 변수 확인
franchise = total_cafe_df[(total_cafe_df['사업장명'].str.contains('공차'))]
franchise

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명,프랜차이즈
2660,공차 서울아산병원점,폐업,2021-02-01,2022-08-01,9.17,서울특별시 송파구 풍납동 388-1 서울아산병원,커피숍,0
7952,(주)공차코리아홍대오버더디쉬점,폐업,2014-10-31,2015-07-01,15.30,서울특별시 마포구 서교동 486번지 서교푸르지오 지201호일부,커피숍,0
8080,공차,폐업,2014-09-18,2017-08-14,18.90,서울특별시 마포구 성산동 515번지 2층일부( P31호),커피숍,0
42568,공차,폐업,2013-11-08,2017-11-08,69.95,서울특별시 동대문구 청량리동 235-4번지 미주상가1층12호,커피숍,0
43018,공차 강남구청역점,폐업,2015-09-23,2019-11-04,53.19,서울특별시 강남구 논현동 241-3번지 강남구청역 에스케이 허브 블루지상1층 101호,커피숍,0
...,...,...,...,...,...,...,...,...
1855690,(주) 공차코리아(대학로점),영업/정상,2013-09-06,NaN,71.34,서울특별시 종로구 명륜2가 24-4,까페,0
1868791,공차 롯데시네마 월드타워점,영업/정상,2014-10-10,NaN,59.57,서울특별시 송파구 신천동 29 7층 엔터테인먼트동,까페,0
1930052,공차 여의도점,영업/정상,2013-04-18,NaN,55.70,서울특별시 영등포구 여의도동 24 한주빌딩1층 남서측일부,까페,0
1940308,공차(GONG CHA),영업/정상,2014-06-25,NaN,39.00,서울특별시 중구 충무로1가 54번지 신세계백화점신관 지하1층,까페,0


In [81]:
#프랜차이즈명으로 레코드 찾기
franchise_total = total_cafe_df[(total_cafe_df['사업장명'].str.contains('이디야')) | (total_cafe_df['사업장명'].str.contains('메가엠지씨')) | (total_cafe_df['사업장명'].str.contains('투썸플레이스')) | (total_cafe_df['사업장명'].str.contains('컴포즈커피')) | (total_cafe_df['사업장명'].str.contains('빽다방')) | (total_cafe_df['사업장명'].str.contains('더벤티')) | (total_cafe_df['사업장명'].str.contains('커피베이')) | (total_cafe_df['사업장명'].str.contains('더벤티'))| (total_cafe_df['사업장명'].str.contains('커피에 반하다')) | (total_cafe_df['사업장명'].str.contains('커피에반하다')) | (total_cafe_df['사업장명'].str.contains('요거프레소')) | (total_cafe_df['사업장명'].str.contains('파스쿠찌')) | (total_cafe_df['사업장명'].str.contains('할리스')) | (total_cafe_df['사업장명'].str.contains('엔제리너스')) | (total_cafe_df['사업장명'].str.contains('디저트39')) | (total_cafe_df['사업장명'].str.contains('디저트 39')) | (total_cafe_df['사업장명'].str.contains('탐앤탐스')) | (total_cafe_df['사업장명'].str.contains('매머드')) | (total_cafe_df['사업장명'].str.contains('감성커피')) | (total_cafe_df['사업장명'].str.contains('달리는커피'))| (total_cafe_df['사업장명'].str.contains('달리는 커피'))| (total_cafe_df['사업장명'].str.contains('카페베네'))| (total_cafe_df['사업장명'].str.contains('셀렉토'))| (total_cafe_df['사업장명'].str.contains('토프레소'))| (total_cafe_df['사업장명'].str.contains('아마스빈'))| (total_cafe_df['사업장명'].str.contains('만랩커피'))| (total_cafe_df['사업장명'].str.contains('더치앤빈 ')) | (total_cafe_df['사업장명'].str.contains('카페게이트')) | (total_cafe_df['사업장명'].str.contains('달콤커피')) | (total_cafe_df['사업장명'].str.contains('커피사피엔스')) | (total_cafe_df['사업장명'].str.contains('커스텀커피'))| (total_cafe_df['사업장명'].str.contains('커피나무 '))| (total_cafe_df['사업장명'].str.contains('커피빈'))| (total_cafe_df['사업장명'].str.contains('스타벅스'))| (total_cafe_df['사업장명'].str.contains('아티제'))| (total_cafe_df['사업장명'].str.contains('폴바셋')) | (total_cafe_df['사업장명'].str.contains('커피스미스')) | (total_cafe_df['사업장명'].str.contains('공차'))]
franchise_idx = franchise_total.index
franchise_idx

Index([   1266,    2511,    2649,    2660,    3507,    3884,    3980,    4050,
          4099,    4391,
       ...
       1937776, 1938554, 1938861, 1940308, 1942166, 1942169, 1942205, 1943589,
       2000127, 2002322],
      dtype='int64', length=5932)

In [82]:
#기존 데이터셋에서 프랜차이즈 레코드 삭제 후 franchise 변수를 붙이기
total_cafe_df2 = total_cafe_df.copy()
total_cafe_df2.drop(franchise_idx, inplace=True)
total_cafe_df2.tail()

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명,프랜차이즈
2061319,구할 구,영업/정상,1970-10-14,NaN,98.66,서울특별시 중구 남산동2가 24-13 (1층2층),까페,0
2061332,갈채,영업/정상,1986-04-25,NaN,19.89,서울특별시 성북구 석관동 71-8,까페,0
2061422,블루씨티,영업/정상,2001-05-23,NaN,85.80,서울특별시 은평구 응암동 84-32 지하1층,까페,0
2061469,유토피아 카페,영업/정상,2014-10-30,NaN,167.32,서울특별시 은평구 갈현동 454-3번지 2층,까페,0
2061604,꽃샘,영업/정상,1983-07-19,NaN,20.80,서울특별시 성북구 길음동 1257-2번지,까페,0


In [83]:
franchise_total['프랜차이즈'] = 1
franchise_total

/var/folders/jc/q2j97l_x5hb7jh99pn7nsbym0000gn/T/ipykernel_628/1640532418.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  franchise_total['프랜차이즈'] = 1


,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명,프랜차이즈
1266,빽다방 빵연구소 연세대SK국제학사점,폐업,2022-06-21,2023-02-28,186.50,서울특별시 서대문구 신촌동 134 연세대학교,커피숍,1
2511,달리는커피 강북구청점,폐업,2022-05-02,2023-05-08,38.00,서울특별시 강북구 수유동 188-4 민스빌,커피숍,1
2649,셀렉토커피 북한산두산위브점,폐업,2021-02-01,2023-06-23,49.08,서울특별시 서대문구 홍은동 467 북한산 두산위브,커피숍,1
2660,공차 서울아산병원점,폐업,2021-02-01,2022-08-01,9.17,서울특별시 송파구 풍납동 388-1 서울아산병원,커피숍,1
3507,이디야 에이스가산포휴점,폐업,2020-12-01,2023-07-11,72.90,서울특별시 금천구 가산동 517-4 에이스 가산 포휴(지식산업센터),커피숍,1
...,...,...,...,...,...,...,...,...
1942169,(주)보나비 아티제 이대목동점,영업/정상,2011-03-29,NaN,152.50,서울특별시 양천구 목동 911-1 이대목동병원 본관1층,까페,1
1942205,(주)보나비 아티제 이대목동2점,영업/정상,2011-02-15,NaN,8.25,서울특별시 양천구 목동 911-1 이대목동병원 병원동2층,까페,1
1943589,달콤커피한남동점,영업/정상,1999-04-14,NaN,169.60,서울특별시 용산구 한남동 634-14,까페,1
2000127,이디야커피(충무아트점),영업/정상,1996-11-06,NaN,59.49,서울특별시 중구 신당동 249-69번지 1층,까페,1


In [84]:
#데이터 합치기
total_cafe_df3 = pd.concat([total_cafe_df2, franchise_total])
total_cafe_df3

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명,프랜차이즈
244,텐퍼센트커피(Tenpercent coffee),폐업,2023-09-18,2023-09-25,53.00,서울특별시 노원구 공릉동 567-4,커피숍,0
331,엔젤리너스커피영등포점,폐업,2023-10-24,2023-10-29,6.60,서울특별시 영등포구 여의도동 84-4 여의도한강공원 이벤트광장,커피숍,0
766,트래쉬버스터즈 스낵바,폐업,2023-10-12,2023-10-26,36.00,서울특별시 강남구 대치동 514 SETEC,커피숍,0
1047,카라멜리에오,폐업,2023-10-26,2023-10-31,12.23,서울특별시 강남구 압구정동 429 현대백화점본점,커피숍,0
1091,담은,폐업,2023-09-14,2023-10-24,56.00,서울특별시 강남구 도곡동 419-6,커피숍,0
...,...,...,...,...,...,...,...,...
1942169,(주)보나비 아티제 이대목동점,영업/정상,2011-03-29,NaN,152.50,서울특별시 양천구 목동 911-1 이대목동병원 본관1층,까페,1
1942205,(주)보나비 아티제 이대목동2점,영업/정상,2011-02-15,NaN,8.25,서울특별시 양천구 목동 911-1 이대목동병원 병원동2층,까페,1
1943589,달콤커피한남동점,영업/정상,1999-04-14,NaN,169.60,서울특별시 용산구 한남동 634-14,까페,1
2000127,이디야커피(충무아트점),영업/정상,1996-11-06,NaN,59.49,서울특별시 중구 신당동 249-69번지 1층,까페,1


### (5)면적변수화 하기
- 면적이 30.0 이하이면 소형(s), 초과하면 일반(l)의 값을 가지는 컬럼 생성

In [85]:
#소형면적 레코드 확인
df_small = total_cafe_df3[total_cafe_df3['소재지면적'] <=30.0]
df_small

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명,프랜차이즈
331,엔젤리너스커피영등포점,폐업,2023-10-24,2023-10-29,6.60,서울특별시 영등포구 여의도동 84-4 여의도한강공원 이벤트광장,커피숍,0
1047,카라멜리에오,폐업,2023-10-26,2023-10-31,12.23,서울특별시 강남구 압구정동 429 현대백화점본점,커피숍,0
1120,카페올립,폐업,2023-09-19,2023-10-11,18.00,서울특별시 강남구 삼성동 159 코엑스,커피숍,0
1161,카페테리아,폐업,2023-10-20,2023-10-30,27.00,서울특별시 강남구 대치동 514 SETEC,커피숍,0
1294,디오니소스 와인샵,폐업,2022-06-17,2022-09-22,3.30,서울특별시 마포구 연남동 564-25 도화엘리시아,커피숍,0
...,...,...,...,...,...,...,...,...
1865821,매머드익스프레스선유도점,영업/정상,2014-09-02,NaN,26.00,서울특별시 영등포구 양평동4가 101-3,까페,1
1935038,매머드익스프레스서울역점,영업/정상,1993-08-06,NaN,28.74,서울특별시 용산구 동자동 11-12 (지상1층),까페,1
1935508,만랩커피 영등포구청점,영업/정상,2012-07-11,NaN,25.45,서울특별시 영등포구 당산동3가 396-11번지,까페,1
1942166,아마스빈 숙대점,영업/정상,2005-02-15,NaN,26.70,서울특별시 용산구 청파동2가 71-127번지 지상1층,까페,1


In [86]:
#소형면적이면 1, 소형면적이 아니면 0 값을 가진다.
total_cafe_df3['소형면적'] = np.where(total_cafe_df3['소재지면적']<= 30.0, 1, 0)
total_cafe_df3

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명,프랜차이즈,소형면적
244,텐퍼센트커피(Tenpercent coffee),폐업,2023-09-18,2023-09-25,53.00,서울특별시 노원구 공릉동 567-4,커피숍,0,0
331,엔젤리너스커피영등포점,폐업,2023-10-24,2023-10-29,6.60,서울특별시 영등포구 여의도동 84-4 여의도한강공원 이벤트광장,커피숍,0,1
766,트래쉬버스터즈 스낵바,폐업,2023-10-12,2023-10-26,36.00,서울특별시 강남구 대치동 514 SETEC,커피숍,0,0
1047,카라멜리에오,폐업,2023-10-26,2023-10-31,12.23,서울특별시 강남구 압구정동 429 현대백화점본점,커피숍,0,1
1091,담은,폐업,2023-09-14,2023-10-24,56.00,서울특별시 강남구 도곡동 419-6,커피숍,0,0
...,...,...,...,...,...,...,...,...,...
1942169,(주)보나비 아티제 이대목동점,영업/정상,2011-03-29,NaN,152.50,서울특별시 양천구 목동 911-1 이대목동병원 본관1층,까페,1,0
1942205,(주)보나비 아티제 이대목동2점,영업/정상,2011-02-15,NaN,8.25,서울특별시 양천구 목동 911-1 이대목동병원 병원동2층,까페,1,1
1943589,달콤커피한남동점,영업/정상,1999-04-14,NaN,169.60,서울특별시 용산구 한남동 634-14,까페,1,0
2000127,이디야커피(충무아트점),영업/정상,1996-11-06,NaN,59.49,서울특별시 중구 신당동 249-69번지 1층,까페,1,0


### (6) 생존여부, 생존기간 변수 생성하기
    - 생존여부 : 영업상태명으로 판단
    - 생존기간 : 파이썬 라이브러리를 사용하여 일단위 생존기간 변수 생성

In [87]:
#폐업 여부 값 확인
total_cafe_df3['영업상태명'].value_counts()

영업상태명
폐업       39157
영업/정상    17859
Name: count, dtype: int64

In [88]:
#폐업이면 1, 영업/정상이면 0
total_cafe_df3['폐업'] = np.where(total_cafe_df3['영업상태명'] == '폐업', 1, 0)
total_cafe_df3

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명,프랜차이즈,소형면적,폐업
244,텐퍼센트커피(Tenpercent coffee),폐업,2023-09-18,2023-09-25,53.00,서울특별시 노원구 공릉동 567-4,커피숍,0,0,1
331,엔젤리너스커피영등포점,폐업,2023-10-24,2023-10-29,6.60,서울특별시 영등포구 여의도동 84-4 여의도한강공원 이벤트광장,커피숍,0,1,1
766,트래쉬버스터즈 스낵바,폐업,2023-10-12,2023-10-26,36.00,서울특별시 강남구 대치동 514 SETEC,커피숍,0,0,1
1047,카라멜리에오,폐업,2023-10-26,2023-10-31,12.23,서울특별시 강남구 압구정동 429 현대백화점본점,커피숍,0,1,1
1091,담은,폐업,2023-09-14,2023-10-24,56.00,서울특별시 강남구 도곡동 419-6,커피숍,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1942169,(주)보나비 아티제 이대목동점,영업/정상,2011-03-29,NaN,152.50,서울특별시 양천구 목동 911-1 이대목동병원 본관1층,까페,1,0,0
1942205,(주)보나비 아티제 이대목동2점,영업/정상,2011-02-15,NaN,8.25,서울특별시 양천구 목동 911-1 이대목동병원 병원동2층,까페,1,1,0
1943589,달콤커피한남동점,영업/정상,1999-04-14,NaN,169.60,서울특별시 용산구 한남동 634-14,까페,1,0,0
2000127,이디야커피(충무아트점),영업/정상,1996-11-06,NaN,59.49,서울특별시 중구 신당동 249-69번지 1층,까페,1,0,0


- 생존일 칼럼 생성
- 생존기간 구하기
- 영업/정상이면 NaN 이므로 데이터셋을 다운로드한 날짜 2023-11-02을 넣어준다.
1. 라이브러리 임포트
2. datetime으로 변환
3. 기간 차 계산

In [89]:
#필요 라이브러리 import
from datetime import datetime

In [90]:
#인허가일자를 datetime으로 변환 확인
date_to_compare = datetime.strptime(total_cafe_df3['인허가일자'][244], "%Y-%m-%d")
print("인허가 날짜 :", date_to_compare)

인허가 날짜 : 2023-09-18 00:00:00


In [91]:
#폐업일자를 datetime으로 확인
date_to_compare2 = datetime.strptime(total_cafe_df3['폐업일자'][244], "%Y-%m-%d")
print("폐업 날짜 :", date_to_compare2)

폐업 날짜 : 2023-09-25 00:00:00


In [92]:
#기간 계산 -> int형으로 반환
date_diff = date_to_compare2 - date_to_compare
print("영업일수 :", date_diff.days)

영업일수 : 7


In [93]:
#영업/정상의 경우 폐업날짜를 NaN에서 데이터를 다운받은 날짜(2023-11-02)로 변경해주기
total_cafe_df3['폐업일'] = total_cafe_df3['폐업일자']
total_cafe_df3.loc[total_cafe_df3['영업상태명'] == '영업/정상', '폐업일'] = "2023-11-02"
total_cafe_df3

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명,프랜차이즈,소형면적,폐업,폐업일
244,텐퍼센트커피(Tenpercent coffee),폐업,2023-09-18,2023-09-25,53.00,서울특별시 노원구 공릉동 567-4,커피숍,0,0,1,2023-09-25
331,엔젤리너스커피영등포점,폐업,2023-10-24,2023-10-29,6.60,서울특별시 영등포구 여의도동 84-4 여의도한강공원 이벤트광장,커피숍,0,1,1,2023-10-29
766,트래쉬버스터즈 스낵바,폐업,2023-10-12,2023-10-26,36.00,서울특별시 강남구 대치동 514 SETEC,커피숍,0,0,1,2023-10-26
1047,카라멜리에오,폐업,2023-10-26,2023-10-31,12.23,서울특별시 강남구 압구정동 429 현대백화점본점,커피숍,0,1,1,2023-10-31
1091,담은,폐업,2023-09-14,2023-10-24,56.00,서울특별시 강남구 도곡동 419-6,커피숍,0,0,1,2023-10-24
...,...,...,...,...,...,...,...,...,...,...,...
1942169,(주)보나비 아티제 이대목동점,영업/정상,2011-03-29,NaN,152.50,서울특별시 양천구 목동 911-1 이대목동병원 본관1층,까페,1,0,0,2023-11-02
1942205,(주)보나비 아티제 이대목동2점,영업/정상,2011-02-15,NaN,8.25,서울특별시 양천구 목동 911-1 이대목동병원 병원동2층,까페,1,1,0,2023-11-02
1943589,달콤커피한남동점,영업/정상,1999-04-14,NaN,169.60,서울특별시 용산구 한남동 634-14,까페,1,0,0,2023-11-02
2000127,이디야커피(충무아트점),영업/정상,1996-11-06,NaN,59.49,서울특별시 중구 신당동 249-69번지 1층,까페,1,0,0,2023-11-02


In [94]:
#datetime 형식으로 변환(str -> datetime)
total_cafe_df3['폐업일']= total_cafe_df3['폐업일'].astype('str')
total_cafe_df3['폐업일'] = pd.to_datetime(total_cafe_df3['폐업일'])
total_cafe_df3['폐업일']

244       2023-09-25
331       2023-10-29
766       2023-10-26
1047      2023-10-31
1091      2023-10-24
             ...    
1942169   2023-11-02
1942205   2023-11-02
1943589   2023-11-02
2000127   2023-11-02
2002322   2023-11-02
Name: 폐업일, Length: 57016, dtype: datetime64[ns]

In [95]:
#datetime 형식으로 변환(str -> datetime)2
total_cafe_df3['영업시작일']= total_cafe_df3['인허가일자'].astype('str')
total_cafe_df3['영업시작일'] = pd.to_datetime(total_cafe_df3['영업시작일'])
total_cafe_df3['영업시작일']

244       2023-09-18
331       2023-10-24
766       2023-10-12
1047      2023-10-26
1091      2023-09-14
             ...    
1942169   2011-03-29
1942205   2011-02-15
1943589   1999-04-14
2000127   1996-11-06
2002322   1997-05-15
Name: 영업시작일, Length: 57016, dtype: datetime64[ns]

In [96]:
#영엉기간 컬럼 생성
total_cafe_df3['영업기간'] = (total_cafe_df3['폐업일'] - total_cafe_df3['영업시작일'])
total_cafe_df3['영업기간'] = total_cafe_df3['영업기간'].dt.days
total_cafe_df3.tail(30)

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명,프랜차이즈,소형면적,폐업,폐업일,영업시작일,영업기간
1858451,할리스커피 한양대시티빌점,영업/정상,2015-01-08,NaN,120.00,서울특별시 성동구 행당동 4-5번지,까페,1,0,0,2023-11-02,2015-01-08,3220
1858831,커피스미스 메트로타워점,영업/정상,2015-03-31,NaN,277.10,서울특별시 중구 남대문로5가 537번지 1층,까페,1,0,0,2023-11-02,2015-03-31,3138
1859301,투썸플레이스 중랑역점,영업/정상,1997-04-23,NaN,247.17,서울특별시 중랑구 상봉동 130-166,까페,1,0,0,2023-11-02,1997-04-23,9689
1865098,매머드 익스프레스 충무로점,영업/정상,2014-12-15,NaN,13.20,서울특별시 중구 충무로3가 24-1,까페,1,1,0,2023-11-02,2014-12-15,3244
1865821,매머드익스프레스선유도점,영업/정상,2014-09-02,NaN,26.00,서울특별시 영등포구 양평동4가 101-3,까페,1,1,0,2023-11-02,2014-09-02,3348
1867327,폴바셋 페럼점,영업/정상,2011-05-30,NaN,155.10,서울특별시 중구 수하동 66번지 페럼타워 1층,까페,1,0,0,2023-11-02,2011-05-30,4539
1868720,서울 만남의 광장 휴게소 할리스매장,영업/정상,2014-05-19,NaN,45.63,서울특별시 서초구 원지동 10-16 1층,까페,1,0,0,2023-11-02,2014-05-19,3454
1868791,공차 롯데시네마 월드타워점,영업/정상,2014-10-10,NaN,59.57,서울특별시 송파구 신천동 29 7층 엔터테인먼트동,까페,1,0,0,2023-11-02,2014-10-10,3310
1869304,이디야,영업/정상,2013-12-27,NaN,110.00,서울특별시 중랑구 상봉동 130-99,까페,1,0,0,2023-11-02,2013-12-27,3597
1928939,(주)보나비 아티제 잠원파스텔점,영업/정상,2010-03-11,NaN,178.30,서울특별시 서초구 잠원동 66-2 파스텔프라자B동 파스텔골프클럽,까페,1,0,0,2023-11-02,2010-03-11,4984


In [97]:
#월단위 영업기간 컬럼 추가
total_cafe_df3['영업기간(월)'] = total_cafe_df3['영업기간'] // 30
total_cafe_df3.tail(10)

,사업장명,영업상태명,인허가일자,폐업일자,소재지면적,소재지전체주소,위생업태명,프랜차이즈,소형면적,폐업,폐업일,영업시작일,영업기간,영업기간(월)
1937776,커피베이서울역점,영업/정상,2015-04-03,NaN,109.61,서울특별시 용산구 동자동 43-54번지 동산빌딩 1층,까페,1,0,0,2023-11-02,2015-04-03,3135,104
1938554,이디야,영업/정상,2006-06-14,NaN,66.12,서울특별시 중구 신당동 241-5,까페,1,0,0,2023-11-02,2006-06-14,6350,211
1938861,폴바셋여의도점,영업/정상,2012-11-13,NaN,189.08,서울특별시 영등포구 여의도동 23-3번지 KTB투자증권빌딩 B동 1층,까페,1,0,0,2023-11-02,2012-11-13,4006,133
1940308,공차(GONG CHA),영업/정상,2014-06-25,NaN,39.00,서울특별시 중구 충무로1가 54번지 신세계백화점신관 지하1층,까페,1,0,0,2023-11-02,2014-06-25,3417,113
1942166,아마스빈 숙대점,영업/정상,2005-02-15,NaN,26.70,서울특별시 용산구 청파동2가 71-127번지 지상1층,까페,1,1,0,2023-11-02,2005-02-15,6834,227
1942169,(주)보나비 아티제 이대목동점,영업/정상,2011-03-29,NaN,152.50,서울특별시 양천구 목동 911-1 이대목동병원 본관1층,까페,1,0,0,2023-11-02,2011-03-29,4601,153
1942205,(주)보나비 아티제 이대목동2점,영업/정상,2011-02-15,NaN,8.25,서울특별시 양천구 목동 911-1 이대목동병원 병원동2층,까페,1,1,0,2023-11-02,2011-02-15,4643,154
1943589,달콤커피한남동점,영업/정상,1999-04-14,NaN,169.60,서울특별시 용산구 한남동 634-14,까페,1,0,0,2023-11-02,1999-04-14,8968,298
2000127,이디야커피(충무아트점),영업/정상,1996-11-06,NaN,59.49,서울특별시 중구 신당동 249-69번지 1층,까페,1,0,0,2023-11-02,1996-11-06,9857,328
2002322,공차 대학로마로니에점,영업/정상,1997-05-15,NaN,161.44,서울특별시 종로구 동숭동 1-104 지상 1층,까페,1,0,0,2023-11-02,1997-05-15,9667,322


## 4. 대학 데이터와 합치기

### (1) 데이터 불러오기

In [98]:
#시-구-동 변수 생성
total_cafe_df3['지번주소1'] = total_cafe_df3['소재지전체주소'].str.split().str[0:3]
total_cafe_df4 = total_cafe_df3[['소재지면적', '프랜차이즈', '폐업', '영업기간(월)', '소형면적']]
total_cafe_df4

,소재지면적,프랜차이즈,폐업,영업기간(월),소형면적
244,53.00,0,1,0,0
331,6.60,0,1,0,1
766,36.00,0,1,0,0
1047,12.23,0,1,0,1
1091,56.00,0,1,1,0
...,...,...,...,...,...
1942169,152.50,1,0,153,0
1942205,8.25,1,0,154,1
1943589,169.60,1,0,298,0
2000127,59.49,1,0,328,0


In [99]:
total_cafe_df4['지번주소'] = total_cafe_df3['지번주소1'].str[0] + " " + total_cafe_df3['지번주소1'].str[1] + " " + total_cafe_df3['지번주소1'].str[2]
total_cafe_df4

/var/folders/jc/q2j97l_x5hb7jh99pn7nsbym0000gn/T/ipykernel_628/3986324604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_cafe_df4['지번주소'] = total_cafe_df3['지번주소1'].str[0] + " " + total_cafe_df3['지번주소1'].str[1] + " " + total_cafe_df3['지번주소1'].str[2]


,소재지면적,프랜차이즈,폐업,영업기간(월),소형면적,지번주소
244,53.00,0,1,0,0,서울특별시 노원구 공릉동
331,6.60,0,1,0,1,서울특별시 영등포구 여의도동
766,36.00,0,1,0,0,서울특별시 강남구 대치동
1047,12.23,0,1,0,1,서울특별시 강남구 압구정동
1091,56.00,0,1,1,0,서울특별시 강남구 도곡동
...,...,...,...,...,...,...
1942169,152.50,1,0,153,0,서울특별시 양천구 목동
1942205,8.25,1,0,154,1,서울특별시 양천구 목동
1943589,169.60,1,0,298,0,서울특별시 용산구 한남동
2000127,59.49,1,0,328,0,서울특별시 중구 신당동


In [100]:
#대학정보 데이터 불러오기
df_univ = pd.read_csv('dong_sum.csv')
df_univ.value_counts()

Unnamed: 0  지번주소              학교명  보유면적       재학생_전체_계  법정동코드     
0           서울특별시 강서구 화곡동     2    99436.0    2141      1150010300    1
25          서울특별시 서대문구 신촌동    1    828528.0   14787     1141011400    1
27          서울특별시 서대문구 홍은동    1    33500.0    6622      1141011800    1
28          서울특별시 서대문구 홍제동    1    4534.0     851       1141011100    1
29          서울특별시 서초구 반포동     1    86673.0    903       1165010700    1
30          서울특별시 서초구 서초동     1    82662.0    1444      1165010800    1
31          서울특별시 성동구 사근동     2    472287.0   23529     1120010600    1
32          서울특별시 성북구 돈암동     1    101874.0   6587      1129010300    1
33          서울특별시 성북구 삼선동2가   1    48726.0    6942      1129011200    1
34          서울특별시 성북구 석관동     1    143768.0   1797      1129013900    1
35          서울특별시 성북구 안암동5가   1    881440.0   20373     1129012500    1
36          서울특별시 성북구 정릉동     2    245351.0   21459     1129013300    1
37          서울특별시 성북구 하월곡동    1    54323.0    6981      1129013600   

### (2) 카페 데이터와 대학 데이터 조인시키기

In [101]:
#데이터 합치기
df_all = pd.merge(left=total_cafe_df4, right=df_univ, how="left", on="지번주소")
df_all = df_all[['소재지면적', '프랜차이즈','소형면적', '폐업', '영업기간(월)', '학교명', '보유면적', '재학생_전체_계']]
df_all

,소재지면적,프랜차이즈,소형면적,폐업,영업기간(월),학교명,보유면적,재학생_전체_계
0,53.00,0,0,1,0,3.0,1049479.0,22578.0
1,6.60,0,1,1,0,NaN,NaN,NaN
2,36.00,0,0,1,0,NaN,NaN,NaN
3,12.23,0,1,1,0,NaN,NaN,NaN
4,56.00,0,0,1,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
57011,152.50,1,0,0,153,NaN,NaN,NaN
57012,8.25,1,1,0,154,NaN,NaN,NaN
57013,169.60,1,0,0,298,NaN,NaN,NaN
57014,59.49,1,0,0,328,NaN,NaN,NaN


In [102]:
#NaN 값 0.0으로 변환
df_all = df_all.fillna(0.0)
df_all

,소재지면적,프랜차이즈,소형면적,폐업,영업기간(월),학교명,보유면적,재학생_전체_계
0,53.00,0,0,1,0,3.0,1049479.0,22578.0
1,6.60,0,1,1,0,0.0,0.0,0.0
2,36.00,0,0,1,0,0.0,0.0,0.0
3,12.23,0,1,1,0,0.0,0.0,0.0
4,56.00,0,0,1,1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
57011,152.50,1,0,0,153,0.0,0.0,0.0
57012,8.25,1,1,0,154,0.0,0.0,0.0
57013,169.60,1,0,0,298,0.0,0.0,0.0
57014,59.49,1,0,0,328,0.0,0.0,0.0


In [103]:
#변수명 변경
df_all.columns = ['area', 'franchise', 'small_area', 'closed', 'period_month', 'univ_count', 'univ_area', 'univ_pop']
df_all

,area,franchise,small_area,closed,period_month,univ_count,univ_area,univ_pop
0,53.00,0,0,1,0,3.0,1049479.0,22578.0
1,6.60,0,1,1,0,0.0,0.0,0.0
2,36.00,0,0,1,0,0.0,0.0,0.0
3,12.23,0,1,1,0,0.0,0.0,0.0
4,56.00,0,0,1,1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
57011,152.50,1,0,0,153,0.0,0.0,0.0
57012,8.25,1,1,0,154,0.0,0.0,0.0
57013,169.60,1,0,0,298,0.0,0.0,0.0
57014,59.49,1,0,0,328,0.0,0.0,0.0


## 5. 최종 데이터 내보내기

In [104]:
df_all.to_csv('./df_all.csv')